In [ ]:
#!/usr/bin/env python3
# boto3>=1.28
import sys
import json
from typing import Optional, Iterable

import boto3
from botocore.exceptions import ClientError, NoCredentialsError, NoRegionError


# -------- Identity helpers --------
def parse_arn_role_or_user(arn: str) -> dict:
    """
    Extracts a concise principal from an STS Arn.
    Examples:
      arn:aws:sts::123456789012:assumed-role/MyRole/cli-123 -> {"type":"assumed-role","name":"MyRole","session":"cli-123"}
      arn:aws:iam::123456789012:user/alice -> {"type":"user","name":"alice"}
      arn:aws:iam::123456789012:role/MyRole -> {"type":"role","name":"MyRole"}
    """
    out = {"raw": arn, "type": "unknown", "name": None, "session": None}
    try:
        _, _, svc, _, _, resource = arn.split(":", 5)
        if svc == "sts" and resource.startswith("assumed-role/"):
            _, role, session = resource.split("/", 2)
            out.update({"type": "assumed-role", "name": role, "session": session})
        elif svc == "iam":
            if resource.startswith("user/"):
                out.update({"type": "user", "name": resource.split("/", 1)[1]})
            elif resource.startswith("role/"):
                out.update({"type": "role", "name": resource.split("/", 1)[1]})
    except Exception:
        pass
    return out


def who_am_i(session: Optional[boto3.session.Session] = None) -> dict:
    """Return account, arn, user/role name, and credential method."""
    session = session or boto3.Session()
    sts = session.client("sts")
    ident = sts.get_caller_identity()
    arn = ident["Arn"]
    parsed = parse_arn_role_or_user(arn)
    creds = session.get_credentials()
    method = getattr(creds, "method", None) if creds else None
    return {
        "account": ident["Account"],
        "user_id": ident["UserId"],
        "arn": arn,
        "principal": parsed,
        "credential_method": method,  # e.g., 'env', 'shared-credentials-file', 'assume-role', 'sso', 'iam-role'
        "region": session.region_name,
        "profile": session.profile_name,
    }


# -------- S3 utilities --------
def s3_list_keys(bucket: str, prefix: str = "", session: Optional[boto3.session.Session] = None) -> Iterable[str]:
    session = session or boto3.Session()
    s3 = session.client("s3")
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            yield obj["Key"]


def s3_write_text(bucket: str, key: str, text: str, session: Optional[boto3.session.Session] = None):
    session = session or boto3.Session()
    s3 = session.client("s3")
    s3.put_object(Bucket=bucket, Key=key, Body=text.encode("utf-8"), ContentType="text/plain; charset=utf-8")


def s3_read_text(bucket: str, key: str, session: Optional[boto3.session.Session] = None) -> str:
    session = session or boto3.Session()
    s3 = session.client("s3")
    obj = s3.get_object(Bucket=bucket, Key=key)
    return obj["Body"].read().decode("utf-8")


# -------- Demo main --------
def main():
    if len(sys.argv) < 2:
        print("Usage: python s3_identity_demo.py <bucket-name> [prefix] [write-key]")
        sys.exit(2)

    bucket = sys.argv[1]
    prefix = sys.argv[2] if len(sys.argv) >= 3 else ""
    write_key = sys.argv[3] if len(sys.argv) >= 4 else "boto3-demo/hello.txt"

    try:
        session = boto3.Session()  # honors AWS_PROFILE, AWS_REGION, SSO, IMDS, etc.

        # 1) Who am I?
        info = who_am_i(session)
        print("# ---- Current AWS identity ----")
        print(json.dumps(info, indent=2))

        # 2) List keys
        print(f"\n# ---- List s3://{bucket}/{prefix} ----")
        any_key = None
        for i, key in enumerate(s3_list_keys(bucket, prefix, session)):
            print(key)
            any_key = any_key or key
            if i > 200:
                print("... truncated ...")
                break
        if not any_key:
            print("(no objects)")

        # 3) Write a file
        payload = "hello from boto3\n"
        s3_write_text(bucket, write_key, payload, session)
        print(f"\nWrote text object: s3://{bucket}/{write_key}")

        # 4) Read it back
        roundtrip = s3_read_text(bucket, write_key, session)
        print("\n# ---- Read back ----")
        print(roundtrip)

    except NoCredentialsError:
        print("No AWS credentials found. Configure env vars, ~/.aws/credentials, SSO, or an instance profile.", file=sys.stderr)
        sys.exit(1)
    except NoRegionError:
        print("No region configured. Set AWS_REGION or configure a default region.", file=sys.stderr)
        sys.exit(1)
    except ClientError as e:
        print(f"AWS error: {e}", file=sys.stderr)
        sys.exit(1)




In [ ]:
import os
account_no = "123456789012"
os.environ['AWS_ROLE_ARN']=f'arn:aws:iam::{account_no}:role/acme-read-bucket-role'
bucket = "ddl-wadkars"



In [ ]:
session = boto3.Session()

In [ ]:
info = who_am_i(session)
print("# ---- Current AWS identity ----")
print(json.dumps(info, indent=2))

In [ ]:
s3_list_keys(bucket)
# 2) List keys
prefix=""
print(f"\n# ---- List s3://{bucket}/{prefix} ----")
any_key = None
for i, key in enumerate(s3_list_keys(bucket, prefix, session)):
    print(key)
    any_key = any_key or key
    if i > 200:
        print("... truncated ...")
        break
    if not any_key:
        print("(no objects)")